In [2]:
from datasets import load_dataset

In [13]:
ds = load_dataset(path="wikimedia/wikipedia", name="20231101.uk", split="train")

In [14]:
ds

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1294720
})

In [15]:
sampled_ds = ds.shuffle(seed=42).select(range(100_000)).select_columns(["text"])

In [16]:
sampled_ds

Dataset({
    features: ['text'],
    num_rows: 100000
})